In [171]:
import networkx as nx
passing_network = nx.DiGraph()
passing_network.add_edges_from(passing_data)

In [172]:
p_b_data = pd.read_csv("playerdata.csv")

In [173]:
#df = df.assign(Percentage = lambda x: (x['Total_Marks'] /500 * 100))
def name_switcher(n):
    l = n.split(" ")
    return str(l[1]) + ", " + str(l[0])
p_b_data['Player'] = p_b_data['Player'].apply(lambda x: name_switcher(x))

In [174]:
p_b_data.to_csv("namesmodifiedplayerdata.csv")

In [175]:
players = p_b_data['Player'].tolist()
pers = p_b_data['PER'].tolist()
asts = p_b_data['AST%'].tolist()
per_data = dict(zip(players, pers))
ast_data = dict(zip(players, asts))

In [176]:
def per_accessor(p):
    try:
        return per_data[p]
    except:
        return 0
    
def ast_accessor(p):
    try:
        return ast_data[p]
    except:
        return 0


G = passing_network

between_centrality = nx.betweenness_centrality(G)


per_centrality = {player: per_accessor(player) for player in G.nodes}

factor=1.0/sum(per_centrality.values())
per_centrality = {k: v*factor for k, v in per_centrality.items() }

ast_centrality = {player: ast_accessor(player) for player in G.nodes}

factor=1.0/sum(ast_centrality.values())
ast_centrality = {k: v*factor for k, v in ast_centrality.items() }

    

# Combine centrality measures
combined_centrality = {player: 0.5*between_centrality[player] + 0.25*per_centrality[player] + 0.25*ast_centrality[player] for player in G.nodes}


In [177]:
# Identify the player with the highest combined centrality
best_player = max(combined_centrality, key=combined_centrality.get)

# Get the neighbors of the best player (potential duo)
duo_candidates = list(G.neighbors(best_player))

# Calculate combined centrality for each potential duo
duo_centrality = {}
for candidate in duo_candidates:
    duo = (best_player, candidate)
    duo_centrality[duo] = combined_centrality[best_player] + combined_centrality[candidate]

# Identify the duo with the highest combined centrality
best_duo = max(duo_centrality, key=duo_centrality.get)

# Display the best duo
print(f"Best Duo: {best_duo} with combined centrality {duo_centrality[best_duo]}")

Best Duo: ('Harper, Jared', 'Temple, Garrett') with combined centrality 0.14315297010231096
